<a href="https://colab.research.google.com/github/Roterdamjr/ControleFinanceiro/blob/main/Controle_Financeiro_Python_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Biblioteca

>



In [1]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 4.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from io import BytesIO
import yfinance as yf
import xlsxwriter

def load_data(excel_filepath):

    df_patrimonio = fn_busca_aba_excel(excel_filepath,'Patrimonio')
    df_alocacao_global = fn_busca_aba_excel(excel_filepath,'Alocacao global')
    df_cadastro_original_acoes = fn_busca_aba_excel(excel_filepath,'Cadastro Original Acoes')
    df_historico = fn_busca_aba_excel(excel_filepath,'Historico')
    df_cadastro_fi = fn_busca_aba_excel(excel_filepath,'Cadastro FI')
    df_cadastro_recebiveis = fn_busca_aba_excel(excel_filepath,'Cadastro Recebiveis')
    df_renda_fixa = fn_busca_aba_excel(excel_filepath,'Renda Fixa')
    df_exterior = fn_busca_aba_excel(excel_filepath,'Exterior')

    df_proporcoes_dh_dp =pd.read_excel(excel_filepath, sheet_name='Proporcoes DH_DP',  engine='openpyxl' )
    proporcao_dh = df_proporcoes_dh_dp['DH'].iloc[0] if not df_proporcoes_dh_dp['DH'].empty else 0.0
    proporcao_dp = df_proporcoes_dh_dp['DP'].iloc[0] if not df_proporcoes_dh_dp['DP'].empty else 0.0

    return (df_patrimonio, df_alocacao_global, df_cadastro_original_acoes, df_historico, df_cadastro_fi,
            df_cadastro_recebiveis, df_renda_fixa,df_exterior,
            proporcao_dh, proporcao_dp)

def fn_busca_aba_excel(excel_filepath,nome_aba ):
    df_aba = pd.read_excel(
        excel_filepath,
        sheet_name=nome_aba,
        engine='openpyxl'
    )

    df_aba = df_aba.dropna(axis=1,how='all')
    df_aba.set_index('Ativo', inplace=True)
    return df_aba

def fn_busca_pm_quantidade(df):

  #recebe dados de uma ativo e deveolve preco médio, quantidade eresultado da vendas
  #df_vendas = pd.DataFrame(columns=['Data', 'Quantidade', 'Preço Compra', 'Preço Venda'])
  preco_medio, qtde_total = 0.0, 0

  for indice, linha in df.iterrows():
      valor = linha['Valor']
      qtde = linha['Quant']
      operacao = linha['Operação']

      if operacao == 'C':
          preco_medio = (preco_medio * qtde_total + valor * qtde) / (qtde_total + qtde)
          qtde_total += qtde
      elif operacao == 'V':
          qtde_total -= qtde
      elif operacao == 'A':
          preco_medio = (preco_medio * qtde_total) / (qtde_total - qtde)
          qtde_total -= qtde
      elif operacao == 'D':
          preco_medio = (preco_medio * qtde_total) / (qtde_total + qtde)
          qtde_total += qtde

  return qtde_total,preco_medio

def fn_calc_metricas(nome_ativo,
                     df_historico,
                     df_alocacao,
                     df_cotacoes ):

  ##############################
  # calcula quantidade, preco_medio, cotacao ,
  # patrimonio, resultado, rentabilidade, alocacao
  ##############################
  cotacao = df_cotacoes[nome_ativo]

  if nome_ativo in df_historico.index:
    df_ativo = df_historico.loc[nome_ativo]
    quantidade, preco_medio = fn_busca_pm_quantidade(df_ativo)

    alocacao = df_alocacao.loc[nome_ativo]['Alocacao']
    patrimonio = quantidade * cotacao
    resultado = patrimonio - quantidade * preco_medio
    rentabilidade = resultado / (quantidade * preco_medio)
  else:
      quantidade = 0
      preco_medio= 0
      alocacao = 0
      patrimonio = 0
      resultado = 0
      rentabilidade = 0

  return [int(quantidade),
          float(round(preco_medio, 2)),
          float(round(cotacao, 2)) ,
          float(round(patrimonio, 2)),
          float(round(resultado, 2)),
          float(round(rentabilidade, 2)),
          float(round(alocacao, 3))]

def fn_gera_carteira( p_df_historico, p_df_cadastro, p_df_cotacoes):
  df_carteira = pd.DataFrame([fn_calc_metricas(ativo, p_df_historico, p_df_cadastro, p_df_cotacoes)
                              for ativo in p_df_cadastro.index],
                          columns=['Quantidade', 'Preço Médio', 'Cotação', 'Patrimônio', 'Resultado', 'Rentabilidade', 'Alocacao'],
                          index=p_df_cadastro.index)

  # Totais
  patrimonio_total, resultado_total = df_carteira['Patrimônio'].sum(), df_carteira['Resultado'].sum()

  return df_carteira

def fn_busca_cotacoes(p_ativos):
  df_cotacoes = pd.DataFrame()
  df_cotacoes = yf.download([at +".SA" for at in p_ativos])['Close'].iloc[-1]
  df_cotacoes.index = df_cotacoes.index.str.replace('.SA', '', regex=False)
  return df_cotacoes

def fn_object_para_numerico(df,nome_coluna):
    df[nome_coluna] = df[nome_coluna].str.replace('.', '', regex=False)
    df[nome_coluna] = df[nome_coluna].str.replace(',', '.', regex=False).astype(float)

def fn_percet_para_numerico(df, nome_coluna):
  coluna_limpa = (
          df[nome_coluna]
          .astype(str)
          .str.replace('%', '', regex=False)
          .str.replace(',', '.', regex=False)
      )

  df['Alocacao'] = pd.to_numeric(coluna_limpa, errors='coerce') / 100

# Planilha e cotacoes

In [4]:
####################################
#     Download da Planilha
####################################
df_patrimonio, df_alocacao_global, df_cadastro_original_acoes, df_historico, df_cadastro_fi, \
            df_cadastro_recebiveis, df_renda_fixa,df_exterior,\
            proporcao_dh, proporcao_dp = load_data('/content/Controle Financeiro.xlsx' )

####################################
#     Tipo tributario
####################################
df_acoes = pd.DataFrame(
    {'Tipo': 'Acao'},
    index=df_cadastro_original_acoes.index.unique()
)
df_acoes.index.name = 'Ativo'

df_fi = pd.DataFrame(
    {'Tipo': 'Fundo Imobiliario'},
    index=df_cadastro_fi.index.unique()
)
df_fi.index.name = 'Ativo'

df_recebiveis = pd.DataFrame(
    {'Tipo': 'Fundo Imobiliario'},
    index=df_cadastro_recebiveis.index.unique()
)
df_recebiveis.index.name = 'Ativo'

df_tipo_tributario = pd.concat([df_acoes, df_fi, df_recebiveis], ignore_index=False)

####################################
#     Download de Cotações
####################################
lista_de_ativos = df_cadastro_original_acoes.index.tolist()
lista_de_ativos.extend(df_cadastro_fi.index.tolist())
lista_de_ativos.extend(df_cadastro_recebiveis.index.tolist())
df_cotacoes = fn_busca_cotacoes(lista_de_ativos)

/tmp/ipython-input-1199865310.py:110: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_cotacoes = yf.download([at +".SA" for at in p_ativos])['Close'].iloc[-1]
[*********************100%***********************]  32 of 32 completed


# Carteira Estruturada Global

In [5]:
df_carteira_global  = pd.DataFrame(columns=['Ativo', 'Patrimonio', 'Alocacao', 'Aportar']).set_index('Ativo')

linhas_a_atualizar = ['Acoes', 'Fundos Imobiliarios','Recebiveis','CDB Dan','CDB Marcia',
                      'Pre-Fixado','Selic','IPCA', 'VGBL Dan','VGBL Marcia', 'Exterior', 'Opcoes']

df_carteira_global = df_carteira_global.reindex(df_carteira_global.index.union(linhas_a_atualizar))
df_carteira_global.loc[linhas_a_atualizar, 'Patrimonio'] = df_patrimonio['Patrimonio'].loc[linhas_a_atualizar]

#### Alocação ###
df_carteira_global.loc['Acoes', 'Alocacao'] = df_alocacao_global ['Alocacao'].loc['Acoes']
df_carteira_global.loc['Fundos Imobiliarios', 'Alocacao'] = df_alocacao_global ['Alocacao'].loc['FII']
df_carteira_global.loc['Recebiveis', 'Alocacao'] = df_alocacao_global ['Alocacao'].loc['Recebiveis']

fator_base = df_alocacao_global.loc['Renda Fixa', 'Alocacao']
indices_rf = df_renda_fixa.index
df_carteira_global.loc[indices_rf, 'Alocacao'] = (
    round(fator_base * df_renda_fixa['Alocacao'],3)
)

df_carteira_global.loc['Exterior', 'Alocacao'] = df_alocacao_global ['Alocacao'].loc['Exterior']
df_carteira_global.loc['Opcoes', 'Alocacao'] = df_alocacao_global ['Alocacao'].loc['Opcoes']

df_carteira_global['Aportar'] = df_carteira_global['Alocacao'] * df_carteira_global['Patrimonio'].sum() - df_carteira_global['Patrimonio']




#  Cadastro ponderado de acoes, FI e Recebiveis

In [6]:
################################################################
# Geracao de df_cadastro_acoes_ponderado
# que se baeia nos pesos do Dica Acoes e Dica Prev
################################################################
df_acoes_dh = df_cadastro_original_acoes.loc[df_cadastro_original_acoes['Tipo'] == 'DH'].copy()
soma_total = df_acoes_dh['Alocacao'].sum()
df_acoes_dh['Calculado'] = df_acoes_dh['Alocacao'] / soma_total * proporcao_dh

df_acoes_dp = df_cadastro_original_acoes.loc[df_cadastro_original_acoes['Tipo'] == 'DP'].copy()
soma_total = df_acoes_dp['Alocacao'].sum()
df_acoes_dp['Calculado'] = df_acoes_dp['Alocacao'] / soma_total * proporcao_dp

df_cadastro_acoes_ponderado = pd.concat([df_acoes_dh, df_acoes_dp]).sort_index()
df_cadastro_acoes_ponderado.drop('Alocacao', axis=1, inplace=True)
df_cadastro_acoes_ponderado.rename(columns={'Calculado': 'Alocacao'}, inplace=True)
df_cadastro_acoes_ponderado['Alocacao'] = df_cadastro_acoes_ponderado['Alocacao'].round(3)

####################################
#  Geração das carteiras de Ações,
#  Fundos imobiliarios e Recebiveis
####################################
df_carteira_acoes = fn_gera_carteira  (df_historico, df_cadastro_acoes_ponderado, df_cotacoes)
df_carteira_acoes['Preco Justo'] = df_cadastro_acoes_ponderado['Preco Justo']

df_carteira_fi = fn_gera_carteira  (df_historico, df_cadastro_fi, df_cotacoes)
df_carteira_recebiveis = fn_gera_carteira  (df_historico, df_cadastro_recebiveis, df_cotacoes)

###### calcula e atualiza coluna Aportar        #####
###### em na carteira de Acoes, FI e Recebiveis #####
tot = df_carteira_acoes['Patrimônio'].sum()
aporte = df_carteira_global['Aportar'].loc['Acoes']
df_carteira_acoes['Aportar'] =  df_carteira_acoes['Alocacao'] * (tot + aporte) - df_carteira_acoes['Patrimônio']

tot = df_carteira_fi['Patrimônio'].sum()
aporte = df_carteira_global['Aportar'].loc['Fundos Imobiliarios']
df_carteira_fi['Aportar'] =  df_carteira_fi['Alocacao'] * (tot + aporte) - df_carteira_fi['Patrimônio']

tot = df_carteira_recebiveis['Patrimônio'].sum()
aporte = df_carteira_global['Aportar'].loc['Recebiveis']
df_carteira_recebiveis['Aportar'] =  df_carteira_recebiveis['Alocacao'] * (tot + aporte) - df_carteira_recebiveis['Patrimônio']

# Imposto de Renda

In [10]:
def fn_vendas_do_ativo(df_hist):
  ######################################################
  # Recebe dados de uma ativo e deveolve data, qtde,valor e preco_medio
  ######################################################

  a_vendas = []
  preco_medio, qtde_total = 0.0, 0

  for indice, linha in df_hist.iterrows():
      valor = linha['Valor']
      qtde = linha['Quant']
      operacao = linha['Operação']
      data= linha['Data']

      if operacao == 'C':
          preco_medio = (preco_medio * qtde_total + valor * qtde) / (qtde_total + qtde)
          qtde_total += qtde
      elif operacao == 'V':
          qtde_total -= qtde
          a_vendas.append([indice,data, qtde,valor,preco_medio])
      elif operacao == 'A':
          preco_medio = (preco_medio * qtde_total) / (qtde_total - qtde)
          qtde_total -= qtde
      elif operacao == 'D':
          preco_medio = (preco_medio * qtde_total) / (qtde_total + qtde)
          qtde_total += qtde

  return a_vendas

def calcular_prejuizo_acumulado(grupo):
    """
    Função que calcula o prejuízo acumulado sequencialmente dentro de um grupo (um Tipo).
    """
    prejuizo_acumulado = 0.0

    for index, linha in grupo.iterrows():
        grupo.loc[index, 'Prejuizo Acumulado'] = prejuizo_acumulado

        if linha['Resultado'] < 0:
            prejuizo_acumulado += linha['Resultado']
        elif linha['Resultado'] > 0:
            if prejuizo_acumulado < 0:
                prejuizo_acumulado += linha['Resultado']

                if prejuizo_acumulado > 0:
                    prejuizo_acumulado = 0.0

    return grupo

In [ ]:
##############################################
# Calculo das Vendas detalhada por ativo
##############################################
a_vendas_total = []

# cria df_vendas chamando fn_vendas() para cada ativo

for nome_ativo in df_tipo_tributario.index:

    if nome_ativo in df_historico.index:
        a_vendas = fn_vendas_do_ativo(df_historico.loc[[nome_ativo]])
        tipo_tributario = df_tipo_tributario.loc[nome_ativo].iloc[0]
        a_vendas_com_tipo = [venda + [tipo_tributario] for venda in a_vendas]

        a_vendas_total.extend(a_vendas_com_tipo)

df_vendas = pd.DataFrame(a_vendas_total, columns=['Ativo', 'Data', 'Qtde Vendida', 'Preço Venda', 'Preço Compra','Tipo'])
df_vendas['Resultado'] = round(
      df_vendas['Qtde Vendida'] * (df_vendas['Preço Venda'] - df_vendas['Preço Compra']),
      2)
df_vendas['Valor Venda'] = df_vendas['Qtde Vendida'] * df_vendas['Preço Venda']

########################################################
# Calculo das Vendas agrupadas por Tipo e mes_ano
########################################################

df_vendas_mensais = df_vendas[['Data', 'Tipo', 'Resultado', 'Valor Venda']].copy()

df_vendas_mensais['mes_ano'] = df_vendas_mensais['Data'].dt.to_period('M')
df_vendas_mensais.drop(['Data'], axis=1, inplace=True)

df_vendas_mensais = df_vendas_mensais.groupby(['Tipo', 'mes_ano']).sum().reset_index()

df_vendas_mensais['Prejuizo Acumulado'] = 0.0

df_vendas_mensais = (
    df_vendas_mensais
    .groupby('Tipo', sort=False)
    .apply(calcular_prejuizo_acumulado)
    .reset_index(drop=True)
)
############################################
# Calculo IR sobre Açoes
############################################

# Base de cálculo: se vendas > 20.000,00 e Resultado- prejuizo acumulado > 0
df_irrf_acao = df_vendas_mensais.loc[df_vendas_mensais['Tipo']=='Acao'].copy()
calculo_cheio = df_irrf_acao['Resultado'] + df_irrf_acao['Prejuizo Acumulado']

df_irrf_acao['Base de Calculo'] = np.where(
    df_irrf_acao['Valor Venda'] > 20000,
    calculo_cheio,
    0
)
df_irrf_acao['Base de Calculo'] = df_irrf_acao['Base de Calculo'].clip(lower=0)
df_irrf_acao['IR'] = df_irrf_acao['Base de Calculo'] * 0.15

############################################
# Calculo IR sobre fundos Imobiliario
############################################

# Base de cálculo: se vendas > 20.000,00 e Resultado- prejuizo acumulado > 0
df_irrf_fi = df_vendas_mensais.loc[df_vendas_mensais['Tipo']=='Fundo Imobiliario'].copy()
df_irrf_fi['Base de Calculo'] = df_irrf_fi['Resultado'] + df_irrf_fi['Prejuizo Acumulado']

df_irrf_fi['Base de Calculo'] = df_irrf_fi['Base de Calculo'].clip(lower=0)
df_irrf_fi['IR'] = df_irrf_fi['Base de Calculo'] * 0.15

df_irrf_fi[['mes_ano','Prejuizo Acumulado','Resultado','Base de Calculo','IR']]


# Exportação

In [ ]:
def fn_exporta(dataframes_dict, nome_arquivo):

    writer = pd.ExcelWriter(nome_arquivo, engine='xlsxwriter')
    workbook = writer.book

    # Definição de Cores e Formatos Base
    COR_CINZA = '#A9A9A9'
    COR_BEGE_CLARO = '#F5F5DC'
    COR_BEGE_PADRAO = '#F0EAD6'
    COR_VERDE_MENTA ='#E9F3EB'
    BORDA = 1

    FORMATO_MOEDA = 'R$ #,##0.00'
    FORMATO_PERCENT = '0.00%'

    base_body_format = {'fg_color': COR_BEGE_PADRAO, 'border': BORDA}

    custom_formats_map = {
        'Ativo': (COR_BEGE_CLARO, None),
        'Patrimonio': (COR_BEGE_PADRAO, FORMATO_MOEDA),
        'Preço Médio': (COR_BEGE_PADRAO, FORMATO_MOEDA),
        'Cotação': (COR_BEGE_PADRAO, FORMATO_MOEDA),
        'Resultado': (COR_BEGE_PADRAO, FORMATO_MOEDA),
        'Aportar': (COR_VERDE_MENTA, FORMATO_MOEDA),
        'Rentabilidade': (COR_BEGE_PADRAO, FORMATO_PERCENT),
        'Alocacao': (COR_BEGE_PADRAO, FORMATO_PERCENT),
    }

    formatos_excel = {}
    for col_name, (cor_fundo, num_format) in custom_formats_map.items():
        fmt_dict = {'fg_color': cor_fundo, 'border': BORDA}
        if num_format:
            fmt_dict['num_format'] = num_format
        formatos_excel[col_name] = workbook.add_format(fmt_dict)

    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': COR_CINZA,
        'border': BORDA
    })

    default_format = workbook.add_format(base_body_format)

    # Novo formato para valores negativos (vermelho)
    red_currency_format = workbook.add_format({'fg_color': COR_BEGE_PADRAO, 'font_color': 'red', 'num_format': FORMATO_MOEDA, 'border': BORDA})
    red_percent_format = workbook.add_format({'fg_color': COR_BEGE_PADRAO, 'font_color': 'red', 'num_format': FORMATO_PERCENT, 'border': BORDA})
    red_telha_currency_format = workbook.add_format({'fg_color': COR_VERDE_MENTA, 'font_color': 'red', 'num_format': FORMATO_MOEDA, 'border': BORDA})

    # Itera sobre cada DataFrame e Cria uma Nova Aba
    for sheet_name, df in dataframes_dict.items():

        df.to_excel(writer, sheet_name=sheet_name, startrow=1, header=False, index=True)

        worksheet = writer.sheets[sheet_name]
        colunas_excel = [df.index.name or 'Ativo'] + list(df.columns.values)

        # Escreva o Cabeçalho e Defina a Largura das Colunas
        for col_num, value in enumerate(colunas_excel):
            worksheet.write(0, col_num, value, header_format)

            if value in ['Ativo', df.index.name]:
                width = 15
            elif value in ['Alocacao', 'Tipo']:
                width = 10
            elif value in ['Quantidade', 'Preço Médio', 'Cotação', 'Resultado']:
                width = 15
            else:
                width = 18

            worksheet.set_column(col_num, col_num, width)

        # Aplique a formatação no corpo da tabela
        num_linhas = len(df)
        for row in range(1, num_linhas + 1):

            for col_num, col_name in enumerate(colunas_excel):

                if col_name == (df.index.name or 'Ativo'):
                    valor = df.index[row - 1]
                else:
                    valor = df.iloc[row - 1][col_name]

                # Determine the base format
                formato = formatos_excel.get(col_name, default_format)

                # Apply conditional formatting for negative values
                if isinstance(valor, (int, float)) and valor < 0:
                    if col_name in ['Resultado', 'Patrimonio', 'Preço Médio', 'Cotação']:
                        formato = red_currency_format
                    elif col_name == 'Aportar':
                        formato = red_telha_currency_format
                    elif col_name in ['Rentabilidade', 'Alocacao']:
                        formato = red_percent_format

                worksheet.write(row, col_num, valor, formato)

    # Salve o arquivo
    writer.close()

In [ ]:
# Crie um dicionário com todos os DataFrames e os nomes desejados para as abas
# Substitua df_carteira_acoes, df_carteira_fi, etc. pelos seus DataFrames reais.
dfs_to_export = {
    'Global': df_carteira_global,
    'Ações': df_carteira_acoes,
    'FII': df_carteira_fi,
    'Recebíveis': df_carteira_recebiveis
}

# Exporta todos para um único arquivo
fn_exporta(dfs_to_export, 'Carteira Global de Investimentos.xlsx')